# Construindo um perfil Wikidata/Scholia para uma organização

Wikidata é uma ferramenta poderosa para a representação do conhecimento. Os projetos WikiCite e Scholia utilizam esse poder para construir redes de conhecimento de metaciência.

Neste template irei demonstrar como podemos construir um perfil na Wikidata para uma organização. Você pode ver o resultado desse processo aqui:

- [Graduate Interdisciplinary Program in Bioinformatics (USP)](https://scholia.toolforge.org/organization/Q102292035)

## Requisitos

- Conhecimento básico sobre a Wikidata
- Conhecimento básico sobre o sistema Scholia
- Python básico

## Criando um item para a instituição/programa

O primeiro passo é criar um item para a instituição acadêmica que você deseja raspar. Você pode fazer isso pela interface web da Wikidata: https://www.wikidata.org/wiki/Special:NewItem 

Você pode ter como exemplo a página do programa de bioinformática da USP: https://www.wikidata.org/wiki/Q102292035.


## Raspando os nomes com selenium

O segundo passo será agregar, numa mesma planilha, todos os nomes da instituição. Isso pode ser feito manualmente, mas seria bastante tedioso.

Se a lista estiver numa página web, as chances são que raspar essa lista será mais rápido (e satisfatório) que adicioná-los manualmente.

Usaremos o browser *headless* (oculto) Selenium para raspar a informação de nosso interesse.

In [41]:
# From https://colab.research.google.com/github/nestauk/im-tutorials/blob/3-ysi-tutorial/notebooks/Web-Scraping/Web%20Scraping%20Tutorial.ipynb#scrollTo=knTVDbU7JeAC
!apt update
!apt install chromium-chromedriver
!pip install selenium

import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

# Enter URL of the page where the names are.
# A good practice is to look for an archived version of the url of interest
# at the Web Archive (https://web.archive.org/)
# In our case, we will use: 
url = "https://web.archive.org/web/20210301225725/https://www.x-meeting.com/events/speakers-2019"

# set option to be headless
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# open it, go to a website, and get results
driver = webdriver.Chrome('chromedriver',options=options)
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,286 kB]
Hit:13 http://ppa.launchpad.

Frequentemente, nomes de docentes são links em uma página ou facilmente identificáveis de alguma outra maneira. Vamos printar o objeto *soup* contendo o código HTML da página e identificar os pesquisadores nela. Vou buscar (via CTRL+F) por Ana Ligia Scott, uma das palestrantes no evento.

In [42]:
soup

<html dir="ltr" lang="en-gb" style="height: auto;"><head><script src="//archive.org/includes/analytics.js?v=cf34f82" type="text/javascript"></script>
<script type="text/javascript">window.addEventListener('DOMContentLoaded',function(){var v=archive_analytics.values;v.service='wb';v.server_name='wwwb-app214.us.archive.org';v.server_ms=3287;archive_analytics.send_pageview({});});</script>
<script charset="utf-8" src="/_static/js/bundle-playback.js?v=36gO9Ebf" type="text/javascript"></script>
<script charset="utf-8" src="/_static/js/wombat.js?v=UHAOicsW" type="text/javascript"></script>
<script type="text/javascript">
  __wm.init("https://web.archive.org/web");
  __wm.wombat("https://www.x-meeting.com/events/speakers-2019","20210301225725","https://web.archive.org/","web","/_static/",
	      "1614639445");
</script>
<link href="/_static/css/banner-styles.css?v=fantwOh2" rel="stylesheet" type="text/css"/>
<link href="/_static/css/iconochive.css?v=qtvMKcIJ" rel="stylesheet" type="text/css"/

Como dito, nomes de docentes costumam ser links, o que também é o caso para esta página.

In [43]:
link_elements = soup.find_all("a", href=True)

for link in link_elements:
    print(link)


<a class=" bd-logo-2" href="https://www.x-meeting.com/events">
<img alt="X-meeting" class=" bd-imagestyles" src="https://www.x-meeting.com/events/templates/xmeeting2019/images/designer/5338106c6809fc1fa859aa21ec8dda83_Logo_XM2019_1.png"/>
</a>
<a class="bd-icon-24 bd-icon " href="#" link-disable="true"></a>
<a href="https://www.x-meeting.com/events/">Home</a>
<a href="https://www.x-meeting.com/events/home-2019"><span>Home</span></a>
<a href="https://bioinfoguy.net/survey/index.php/946689?lang=en" target="_blank"><span>X-meeting followup</span></a>
<a href="https://www.x-meeting.com/events/"><span>Previous X-meeting</span></a>
<a href="https://www.x-meeting.com/events/reg-2019/cancellation-policy"><span>Registration</span></a>
<a href="https://docs.google.com/document/d/1JvL3m5FW097oFBWpOLLLZrh4mdN9oCVlaHmlO-TJyZQ/edit?usp=sharing"><span>BR-SCS - Program at a glance</span></a>
<a href="https://www.x-meeting.com/events/program-2019"><span>Scientific program</span></a>
<a href="http://dri

Nesse exemplo, todos os nomes de palestrantes possuem "speakers-2019/1" na URL, portanto podemos utilizar isso para identificá-los.

In [44]:
unique_url_section = "speakers-2019/1"

link_elements = soup.find_all("a", href=True)

for link in link_elements:
    href = (link.get('href'))
    if unique_url_section in href:
      print(link.text.strip())

Ana Ligia Scott
Aristóteles Goes Neto
Marta Giovanetti
Benilton Carvalho
Sabrina de Azevedo Silveira
Dr. Bart Weimer
Támas Korcsmáros
Dr. Gustavo Fioravanti
Tulio de Oliveira
Hugo Verli


Vamos criar uma lista com todos os nomes

In [45]:
link_elements = soup.find_all("a", href=True)

people = []
for link in link_elements:
    href = (link.get('href'))
    if unique_url_section in href:
      people.append(link.text.strip())

Agora nós temos uma lista dos nomes que utilizaremos na Wikidata.

O próximo passo é comparar os nomes a identificadores da Wikidata. Essa tarefa será mais fácil se tirarmos os títulos dos nomes. Por exemplo, na lista de autores do X-Meeting 2019 existem alguns nomes com o prefixo "Dr.", vamos removê-los.

In [46]:
people = [string.replace("Dr. ", "") for string in people]

print(people)

['Ana Ligia Scott', 'Aristóteles Goes Neto', 'Marta Giovanetti', 'Benilton Carvalho', 'Sabrina de Azevedo Silveira', 'Bart Weimer', 'Támas Korcsmáros', 'Gustavo Fioravanti', 'Tulio de Oliveira', 'Hugo Verli']


O próximo passo envolve curadoria manual dos itens para cada pessoa na Wikidata.

Para isso, utilizaremos pandas e gspread para mandar essa informação para um arquivo do Google Planilhas.

In [47]:
df = pd.DataFrame({"names":people})

df

,names
0,Ana Ligia Scott
1,Aristóteles Goes Neto
2,Marta Giovanetti
3,Benilton Carvalho
4,Sabrina de Azevedo Silveira
5,Bart Weimer
6,Támas Korcsmáros
7,Gustavo Fioravanti
8,Tulio de Oliveira
9,Hugo Verli


In [48]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [49]:
spreadsheet_name = "X Meeting 2019 - Tutorial"

Essa porção do código só deve rodar uma vez, ela cria a planilha para a curadoria manual.


In [50]:
sh = gc.create(spreadsheet_name)

In [51]:
sh = gc.open(spreadsheet_name)
worksheet = sh.sheet1
print("https://docs.google.com/spreadsheets/d/%s" % sh.id)

https://docs.google.com/spreadsheets/d/1fN56POnrFgtez7WduirLydhVoJL9dGp36pvJIcOcEBM


Agora você deve clicar no link acima e realizar os seguintes passos:
- Instalar o "Wikidata and Wikipedia Tools Add On"
- Clique em Extensões -> Complementos -> Instalar complementos e busque pelo nome acima.

Após ter instalado a extensão, rode o código abaixo para preencher a planilha.

In [58]:
worksheet.update_cell(1, 1, df.columns[0])
worksheet.update_cell(1, 2, "wikidata_id")
worksheet.update_cell(1, 3, "blank")
worksheet.update_cell(1, 4, "wikidata_description")


for i, row in df.iterrows():
  worksheet.update_cell(i+2, 1, row[0])
  worksheet.update_cell(i+2, 2, "=WIKIDATASEARCH(A"+ str(i+2)+")")
  worksheet.update_cell(1, 3, "")
  worksheet.update_cell(i+2, 4, "=WIKIDATADESCRIPTIONS(B"+ str(i+2)+")")



Antes de prosseguir, confira que os valores foram automaticamente adicionados através da Wikidata e preencha qualquer célula vazia.

Muitas vezes os pesquisadores já estarão na Wikidata, mas alguns sobrenomes são omitidos - ou há alguma variação - o que pode complicar as coisas.

Tenha cuidado com nomes muito comuns: Pode haver mais de um pesquisador com o nome *exatamente* igual.

Em seguida, você está pronto para a parte dois do tutorial: 

## Integrar os dados à Wikidata



In [59]:
list_of_lists = worksheet.get_all_values()

In [60]:
df = pd.DataFrame(list_of_lists)

In [61]:
new_header = df.iloc[0, [0, 1, 3]] # Use a primeira linha como cabeçalho
# Use os dados menos a primeira linha e apenas as colunas contendo informação
df = df.iloc[1:,[0, 1, 4]] 
df.columns = new_header

In [62]:
df

,0,wikidata_id,wikidata_description
1,names,Carregando…,
2,Ana Ligia Scott,Carregando…,
3,Aristóteles Goes Neto,Carregando…,
4,Marta Giovanetti,Carregando…,
5,Benilton Carvalho,Carregando…,
6,Sabrina de Azevedo Silveira,Carregando…,
7,Bart Weimer,Q42290934,researcher
8,Támas Korcsmáros,Q85874612,researcher
9,Gustavo Fioravanti,Q57118765,researcher
10,Tulio de Oliveira,Q30650397,Brazilian bioinformatics and public health sch...


Agora iremos criar [QuickStatements](https://www.wikidata.org/wiki/Help:QuickStatements/pt-br), uma forma de fazer edições em lotes na Wikidata. Atenção que para usar os QuickStatements, sua conta na Wikidata deve ser [autoconfirmada](https://www.wikidata.org/wiki/Wikidata:Autoconfirmed_users/pt-br)

In [63]:
from time import gmtime, strftime

def print_quickstatements(df,
                          prop,
                          url,
                          subj_column="blank",
                          value_column="blank",
                          subj="from_df",
                          value="from_df",
                          date="today",
                          qualifier="none",
                          qualifier_value="none"):
    """
    From a dataframe, print a series of statements 
    for Wikidata using Quickstatements V1 syntax. 
    
    Args:
        df: a pandas dataframe with ids in one column
        
        subj: the subject to state (same for all). Leave "from_df" to obtain from table. 

        prop: the property to state (same for all)
        
        value: the value to state (same for all).  Leave "from_df" to obtain from table. 
        
        url: the reference url
               
        date: the date of retrieval of the info in ISO 8601 format.
        Defaults to magic word "today".
        
        qualifier: A qualifier for the statement. Defaults to "P2868" (subject has role )
        
        role: the role in the organization. Defaults to 
    
    """
    for i, row in df.iterrows():
    
        if subj == "from_df":
          s = row[subj_column]
        else:
          s = subj
        
        p = "|" + prop + "|"
        
        if value =="from_df":
          o = row[value_column]
        else:
          o = value
          
        rp1 = "|S854|"
        rp2 = "|S813|"
        ro1 = '"' + url + '"'


        if date == "today":
            ro2 = strftime("+%Y-%m-%dT00:00:00Z/11", gmtime())
        else:
            ro2 = date
            
            
        if qualifier == "none":
            print(s+p+o+rp1+ro1+rp2+ro2)
        else:
            qp1 = "|" + qualifier + "|"
            qo1 = qualifier_value 
            print(s+p+o+qp1+qo1+rp1+ro1+rp2+ro2)



### Configurar as opções para integração na Wikidata

In [64]:
subject = "Q106812598" # o ID para a conferência
prop = "P823" # A propriedade 'palestrante'
value = "from_df"
qualifier = "none"
subj_column = "blank"
value_column = "wikidata_id"
qualifier_value = "none"

# URL para referência era
url=url
print_quickstatements(df,
                      subj = subject,
                      prop=prop,
                      value=value,
                      subj_column = "blank",
                      value_column = "wikidata_id",
                      url=url,
                      date="today",
                      qualifier=qualifier,
                      qualifier_value=qualifier_value)


Q106812598|P823|Carregando…|S854|"https://web.archive.org/web/20210301225725/https://www.x-meeting.com/events/speakers-2019"|S813|+2022-06-06T00:00:00Z/11
Q106812598|P823|Carregando…|S854|"https://web.archive.org/web/20210301225725/https://www.x-meeting.com/events/speakers-2019"|S813|+2022-06-06T00:00:00Z/11
Q106812598|P823|Carregando…|S854|"https://web.archive.org/web/20210301225725/https://www.x-meeting.com/events/speakers-2019"|S813|+2022-06-06T00:00:00Z/11
Q106812598|P823|Carregando…|S854|"https://web.archive.org/web/20210301225725/https://www.x-meeting.com/events/speakers-2019"|S813|+2022-06-06T00:00:00Z/11
Q106812598|P823|Carregando…|S854|"https://web.archive.org/web/20210301225725/https://www.x-meeting.com/events/speakers-2019"|S813|+2022-06-06T00:00:00Z/11
Q106812598|P823|Carregando…|S854|"https://web.archive.org/web/20210301225725/https://www.x-meeting.com/events/speakers-2019"|S813|+2022-06-06T00:00:00Z/11
Q106812598|P823|Q42290934|S854|"https://web.archive.org/web/2021030122

Para realizar suas edições, copie a saída da célula acima, vá em https://quickstatements.toolforge.org/#/batch, insira os quickstatements e clique em "Importar comandos da Versão 1"